In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import os
from tqdm import tqdm
import cv2
from glob import glob
import numpy as np
import pandas as pd
import PIL
import urllib
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset,DataLoader

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [ ]:
os.chdir('C:/Users/user/Desktop/LG comp/')

데이터 불러오기

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
    
device

In [ ]:
train_csv = pd.read_csv('./data/train.csv')
test_csv = pd.read_csv('./data/test.csv')

In [ ]:
train_csv.head()

In [ ]:
test_csv.head()

In [ ]:
img_size= 256

In [ ]:
train_input_files =  glob(f'./data/train_input_img_{img_size}/*.npy')
train_label_files = glob(f'./data/train_label_img_{img_size}/*.npy')

val_input_files = glob(f'./data/val_input_img_{img_size}/*.npy')
val_label_files = glob(f'./data/val_label_img_{img_size}/*.npy')

In [ ]:
len(train_input_files), len(train_label_files)

#참조
https://blog.promedius.ai/pytorch_dataloader_1/

AlbumentationsDataset

In [ ]:
import albumentations
import torch
import albumentations.pytorch
import torchvision

# from albumentations.torch import ToTensor 
print(albumentations.__version__)

In [ ]:
#원본 이미지로 불러올때 사용
class AlbumentationsDataset(torch.utils.data.Dataset):
    def __init__(self, x_dir, y_dir,transform=None):
        super().__init__()
        self.transforms = transform
        self.x_img = x_dir
        self.y_img = y_dir   

    def __len__(self):
        return len(self.x_img)

    def __getitem__(self, idx):
        x_img = self.x_img[idx]
        y_img = self.y_img[idx]
        # Read an image with OpenCV
        x_img = cv2.imread(x_img)
        y_img =cv2.imread(y_img)
        x_img= cv2.cvtColor(x_img, cv2.COLOR_BGR2RGB)
        y_img= cv2.cvtColor(y_img, cv2.COLOR_BGR2RGB)

        if self.transforms:
            augmented = self.transforms(image=x_img,target_image=y_img)
            image = augmented['image']
            labeled = augmented['target_image']

        return image, labeled 

In [ ]:
#Crop 이미지로 불러올때 사용(baseline은 이미지를 크롭해서 npy format으로 저장해 불러옴)
class AlbumentationsDataset(torch.utils.data.Dataset):
    def __init__(self, x_dir, y_dir,transform=None):
        super().__init__()
        self.transforms = transform
        self.x_img = x_dir
        self.y_img = y_dir  
    def __len__(self):
        return len(self.x_img)

    def __getitem__(self, idx):
        x_img = self.x_img[idx]
        x_img = np.load(x_img)
        x_img = x_img.astype(np.float32)
        
        y_img = self.y_img[idx]
        y_img = np.load(y_img)
        y_img = y_img.astype(np.float32)

        if self.transforms:
            augmented = self.transforms(image=x_img,target_image=y_img)
            image = augmented['image']
            labeled = augmented['target_image']

        return image, labeled

In [ ]:
aug = albumentations.Compose([
      albumentations.Resize(256, 256),
      albumentations.HorizontalFlip(),
      albumentations.RandomRotate90(),
      albumentations.VerticalFlip(),
      albumentations.MedianBlur(blur_limit=5),
      albumentations.MotionBlur(),
      albumentations.pytorch.transforms.ToTensorV2()
      ],additional_targets={'target_image':'image'})

In [ ]:
train_dataset = AlbumentationsDataset(train_input_files,train_label_files,transform=aug)
vali_dataset = AlbumentationsDataset(val_input_files,val_label_files,transform=aug2)

In [ ]:
train_dataloader=torch.utils.data.DataLoader(train_dataset,batch_size=6,shuffle=False)
vali_dataloader=torch.utils.data.DataLoader(vali_dataset,batch_size=6,shuffle=False)

In [ ]:
images,labels = next(iter(train_dataloader))

print(images[0].shape)

x=images[2].numpy()
xx=np.transpose(x,(1,2,0)) #트렌스포즈 해주는 이유는 impose는 채널이 맨 뒤로감
y=labels[2].numpy()
yy=np.transpose(y,(1,2,0))

plt.subplot(1,2,1)
plt.imshow(xx)
plt.subplot(1,2,2)
plt.imshow(yy)
plt.show()

모델은 오픈소스를 사용했으나 출처를 못 찾겠어서 생략합니다

In [ ]:
sum([param.nelement() for param in model.parameters()])

In [ ]:
criterion = nn.L1Loss()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from torch.utils.data.sampler import SubsetRandomSampler
kf = KFold(n_splits=5, shuffle=True)

In [ ]:
import torch.cuda.amp as amp
n_epochs=5
batch_size=8

scaler = amp.GradScaler()
valid_loss_min = np.Inf
train_loss = torch.zeros(n_epochs)
valid_loss = torch.zeros(n_epochs)

for fold,( train_ind, valid_ind) in enumerate(kf.split(train_dataset)):
    
    print('Starting fold = ', fold)

    train_sampler_kfold = SubsetRandomSampler(train_ind)
    valid_sampler_kfold = SubsetRandomSampler(valid_ind)
    
    train_loader_kfold = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler_kfold)
    valid_loader_kfold = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=valid_sampler_kfold)

    model = ResUnet(3).cuda()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.to(device)
    for e in np.arange(n_epochs): 
        print('Starting epoch = ', e)
        
        # Training
        model.train()
        for data,labels in tqdm(train_loader_kfold):
            data = data.to(device=device, dtype=torch.float)
            labels = labels.to(device=device, dtype=torch.float)
            optimizer.zero_grad()
            with amp.autocast():
                output = model(data)
                loss = criterion(output, labels)
            scaler.scale(loss).backward()
            scaler.update()
            train_loss[e] += loss.item()
            
        train_loss[e] /= len(train_loader_kfold)


        # Validation
        model.eval()
        with torch.no_grad():
            for data, labels in tqdm(valid_loader_kfold):
                data = data.to(device=device, dtype=torch.float)
                labels = labels.to(device=device, dtype=torch.float)
                
                with amp.autocast():
                    output = model(data)
                    loss = criterion(output, labels)
                    
                valid_loss[e] += loss.item()
            valid_loss[e] /= len(valid_loader_kfold)
    
    # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(e, train_loss[e], valid_loss[e]))

    
    # save model if validation loss has decreased
        if valid_loss[e] <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss[e]))
            torch.save(model.state_dict(), 'model_'+str(fold)+'.pt')
            valid_loss_min = valid_loss[e]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_loss)
plt.plot(valid_loss)

In [ ]:
model.load_state_dict(torch.load('../model_1.pt'))

In [ ]:
test_input_files = '../test_input_img/'+test_csv['input_img']

In [ ]:
images,labels = next(iter(test_dataloader))
print(images.shape)

x=images[0].numpy()
xx=np.transpose(x,(1,2,0))
y=labels[0].numpy()
yy=np.transpose(y,(1,2,0))

plt.subplot(1,2,1)
plt.imshow(xx)
plt.subplot(1,2,2)
plt.imshow(yy)
plt.show()

이 부분부터 baseline code와 동일

In [ ]:
def predict(img_paths, stride=32, batch_size=128):
    results = []
    for img_path in img_paths:
        img = cv2.imread(img_path)
        img = img.astype(np.float32)/255
        crop = []
        position = []
        batch_count = 0

        result_img = np.zeros_like(img)
        voting_mask = np.zeros_like(img)

        for top in tqdm(range(0, img.shape[0], stride)):
            for left in range(0, img.shape[1], stride):
                piece = np.zeros([img_size, img_size, 3], np.float32)
                temp = img[top:top+img_size, left:left+img_size, :]
                piece[:temp.shape[0], :temp.shape[1], :] = temp
                crop.append(piece)
                position.append([top, left])
                batch_count += 1
                if batch_count == batch_size:
                    crop = np.array(crop)
                    pred = model(crop)*255
                    crop = []
                    batch_count = 0
                    for num, (t, l) in enumerate(position):
                        piece = pred[num]
                        h, w, c = result_img[t:t+img_size, l:l+img_size, :].shape
                        result_img[t:t+img_size, l:l+img_size, :] += piece[:h, :w]
                        voting_mask[t:t+img_size, l:l+img_size, :] += 1
                    position = []
        
        result_img = result_img/voting_mask
        result_img = result_img.astype(np.uint8)
        results.append(result_img)
        
    return results

In [ ]:
import math
def rmse_score(true, pred):
    score = math.sqrt(np.mean((true-pred)**2))
    return score

def psnr_score(true, pred, pixel_max):
    score = 20*np.log10(pixel_max/rmse_score(true, pred))
    return score

In [ ]:
img_size =256
test_result = predict(train_input_files[:5])

In [ ]:
for i, (input_path, label_path) in enumerate(zip(train_input_files[:10], train_label_files[:10])):
    input_img = cv2.imread(input_path)
    input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)

    targ_img = cv2.imread(label_path)
    targ_img = cv2.cvtColor(targ_img, cv2.COLOR_BGR2RGB)
    
    pred_img = result[i]
    pred_img=np.transpose(pred_img,(1,2,0))
    
    plt.figure(figsize=(20,10))
    plt.subplot(1,3,1)
    plt.imshow(input_img)
    plt.title('input_img', fontsize=10)
    plt.subplot(1,3,2)
    plt.imshow(pred_img)
    plt.title('output_img', fontsize=10)
    plt.subplot(1,3,3)
    plt.imshow(targ_img)
    plt.title('target_img', fontsize=10)
    plt.show()
    print('input PSNR :', psnr_score(input_img.astype(float), targ_img.astype(float), 255))
    print('output PSNR :', psnr_score(pred_img[i].astype(float), targ_img.astype(float), 255), '\n')

In [ ]:
import zipfile
def make_submission(result):
    os.makedirs('submission', exist_ok=True)
    os.chdir("./submission/")
    sub_imgs = []
    for i, img in enumerate(result):
        path = f'test_{20000+i}.png'
        cv2.imwrite(path, img)
        sub_imgs.append(path)
    submission = zipfile.ZipFile("submission.zip", 'w')
    for path in sub_imgs:
        submission.write(path)
    submission.close()

In [ ]:
make_submission(test_result)